# 使用 Numpy 中的 as_strided 与 tensordot 实现卷积层

2019 年 5 月 19 日

参考资料:

+ [卷积算法另一种高效实现，as_strided详解](https://zhuanlan.zhihu.com/p/64933417) 介绍 as_strided 函数
+ [numpy.tensordot实现CNN卷积层](https://zhuanlan.zhihu.com/p/65023647) 介绍 tensordot 函数
+ [https://github.com/sebgao/cTensor](https://github.com/sebgao/cTensor) cTensor, 基本参考这个实现
+ [如何理解空洞卷积（dilated convolution）？](https://www.zhihu.com/question/54149221) 看 GIT 了解 dilation

推荐资料:
+ [Convnet: Implementing Convolution Layer with Numpy](https://wiseodd.github.io/techblog/2016/07/16/convnet-conv-layer/) 可以看看, 使用循环实现 im2col, 然后再矩阵乘法
+ [Part 1: Backpropagation for Convolution with Strides](https://medium.com/@mayank.utexas/backpropagation-for-convolution-with-strides-8137e4fc2710) 超多动图, 对于含 stride 的卷积的反向传播介绍的非常详细
+ [Part 2: Backpropagation for Convolution with Strides](https://medium.com/@mayank.utexas/backpropagation-for-convolution-with-strides-fb2f2efc4faa) 上篇博客的续篇
 
从 "推荐资料" 中的最后两篇博文, 精炼的总结出了反向传播的特点:

+ It turns out that the Backpropagation operation is identical to a stride = 1 Convolution of a padded, dilated version of the output gradient tensor with a flipped version of the filter!
+ It turns out that the backpropagation operation is identical to a stride = 1 convolution operation of the input tensor with a dilated version of the output gradient tensor!

之后在代码中体会.

## 卷积层的反向传播

反向传播主要是进行**梯度更新**和**误差传播**.

对于误差传播, 等价于先对输出误差 eta 进行 padding 以及 dilation, 然后再与翻转后的权重进行卷积. 

而对于权重的梯度更新, 等价于先对输入进行 dilation, 然后再与输出误差 eta 进行卷积.

首先实现一些辅助函数, 比如 padding, dilation, 以及 unwrap_padding, rotate_kernel.

In [1]:
import numpy as np

def make_padding(input, padding=(0, 0)):
    if padding == (0, 0):
        return input
    B, C, H, W = input.shape
    pad = np.zeros((B, C, H + 2 * padding[0], W + 2 * padding[1]))
    pad[..., padding[0]:-padding[0], padding[1]:-padding[1]] = input
    return pad

def make_dilation(input, dilation=(1, 1)):
    if dilation == (1, 1):
        return input
    
    B, C, H, W = input.shape
    p, q = dilation
    oh, ow = p * (H - 1) + 1, q * (W - 1) + 1
    pad = np.zeros((B, C, oh, ow))
    pad[..., ::p, ::q] = input
    return pad

def unwrap_padding(input, padding=(0, 0)):
    if padding == (0, 0):
        return input
    p, q = padding
    return input[..., p:-p, q:-q]


def rotate_kernel(kernel):
    return kernel[..., ::-1, ::-1]

## 使用 as_strided 实现 im2col

关于这个方法的实现, 可以参考 [numpy 中的 as_strided 函数](./numpy%20中的%20as_strided%20函数.ipynb) (就是当前目录下的 `numpy 中的 as_strided 函数.ipynb` 文件)

然而下方实现中需要注意的是, 有这样的判断:

```python
assert int(oh) == oh and int(ow) == ow, 'conv2d not aligned'
```

这就希望在前向传播的时候:

$$
    oh = \frac{h + 2p - \left(d\times\left(kh - 1\right) + 1\right)}{s} + 1
$$

$h + 2p - \left(d\times\left(kh - 1\right) + 1\right)$ 能够整除 $s$, 否则抛出异常. 之所以这样做, 是考虑在无法整除时, 反向传播的代码就更复杂了, 要考虑额外的情况. 为了简便期间, 增加这个约束. **以后我再研究如何处理无法整除的情况**.

In [2]:
def im2col(input, ksize, stride=(1, 1), dilation=(1, 1), writeable=False):
    """
    使用 np.lib.stride_tricks.as_strided 实现 im2col, 关键是确认输出结果的 shape 和 strides 两个参数.
    input 应该在输入这个函数之前进行 padding
    """
    B, C, H, W = input.shape
    kh, kw = ksize
    
    istrides = input.strides
    ostrides = list(istrides + istrides[-2:])
    oh = (H - (dilation[0] * (kh - 1) + 1)) / stride[0] + 1
    ow = (W - (dilation[1] * (kw - 1) + 1)) / stride[1] + 1
    assert int(oh) == oh and int(ow) == ow, 'conv2d not aligned'
    oh = int(oh)
    ow = int(ow)
    ostrides[2] *= stride[0]
    ostrides[3] *= stride[1]
    ostrides[4] *= dilation[0]
    ostrides[5] *= dilation[1]
    
    return np.lib.stride_tricks.as_strided(input, shape=(B, C, oh, ow, kh, kw),
                                          strides=ostrides,
                                          writeable=writeable)

a = np.arange(9).reshape(1, 1, 3, 3)
im2col(a, (2, 2), stride=(2, 2), dilation=(2, 2))

array([[[[[[0, 2],
           [6, 8]]]]]])

## 卷积层的前向传播

也考虑了 stride 的情况.


In [3]:
def conv(input, kernel, padding=(0, 0), stride=(1, 1), dilation=(1, 1)):
    input = make_padding(input, padding)
    input = make_dilation(input, dilation)
    
    x_col = im2col(input, kernel.shape[-2:], stride)
    # tensordot 的结果 shape 为 (B, oh, ow, oC)
    return np.tensordot(x_col, kernel, axes=[(1, 4, 5), (0, 2, 3)]).transpose(0, 3, 1, 2)

input = np.arange(9).reshape(1, 1, 3, 3)
kernel = np.ones((1, 1, 2, 2))
out_conv = conv(input, kernel)
print(repr(out_conv))

array([[[[ 8., 12.],
         [20., 24.]]]])


## 卷积层的反向传播

+ 首先, 对于卷积层的反向传播, 需要输入特征 input, 滤波器 kernel, 输出误差 eta, 以及输入时的各种信息, 如 padding, dilation, stride 之类.

### 对于误差传播

+ 结果是权重和 eta 的卷积, 然而考虑到 padding, stride 之类的, 需要先对 eta 进行 dilation, **需要注意的 dilation 的参数值等于 stride**, 然后再进行 padding, padding 的大小为 `(kh - 1, kw - 1)`. 然后再让 kernel 与 eta 进行卷积. 最后对结果进行 `unwrap_padding`.

下面来分析每个步骤得到的特征图的大小:

$$
\begin{aligned}
\text{ieta:} &\quad oh \\
\text{dilation:} &\quad nh = (oh - 1) * s + 1 \\
\text{padding:} &\quad H = nh + 2 * (kh - 1) \\
\text{im2col:} &\quad r = H - kh + 1
\end{aligned}
$$

$r$ 就是 `x_col` 的大小 (height 方向). 再将 $r$ 进行运算: 

注意有 (前向传播):  $(oh - 1) * s + 1 = h + 2p - (kh -1)$ 成立

$$
\begin{aligned}
r &= H - (kh - 1) \\
&= nh + 2 * (kh - 1) - (kh - 1) \\
&= (oh - 1) * s + 1 + (kh - 1)  \\
&= h + 2p - (kh -1) + (kh - 1) \\
&= h + 2p
\end{aligned}
$$

因此最后 `x_col` 的 `shape=(N, oC, h+2p, w+2p, kh, kw)`, 与 kernel (`shape=(iC, oC, kh, kw)`) 进行卷积, 得到 `input_grad` 的 `shape=(N, iC, h+2p, w+2p)`, 然后进行 `unwrap_padding` 即得到和输入 input 的 shape 相同的梯度.

### 对于梯度更新

分析也同理, 

+ 首先需要对输入进行 padding, (因为前向传播时是有 padding 过的), 然后重点在 `im2col` 时要对 input 进行 dilation, 值为 `stride`. 可以使用前面的分析:

$$
\begin{aligned}
\text{padding:} &\quad H = h + 2p \\
\text{im2col:} &\quad r = H - s * (oh - 1) + 1
\end{aligned}
$$

$r$ 进一步化简为:

注意有 (前向传播):  $(oh - 1) * s + 1 = h + 2p - (kh -1)$ 成立

$$
\begin{aligned}
r &= h + 2p - s * (oh - 1) + 1 \\
  &= kh
\end{aligned}
$$

最后 `x_col` 的 `shape=(N, iC, kh, kw, oh, ow)` 与 eta (`shape=(N, oC, oh, ow)`) 进行卷积, 得到 `kernel_grad` 的 `shape=(iC, oC, kh, kw)`.

In [4]:
def backward_conv(input, kernel, eta, padding=(0, 0), stride=(1, 1), dilation=(1, 1)):
    kernel_grad = np.zeros_like(kernel)
    kh, kw = kernel.shape[-2:]
    s, _ = stride
    p, _ = padding
    d, _ = dilation
    oh, ow = eta.shape[-2:]
    
    ## 误差传播
    ieta = eta.copy()
    ieta = make_dilation(ieta, stride)
    ieta = make_padding(ieta, ((kh - 1), (kw - 1)))
    x_col = im2col(ieta, (kh, kw))
    input_grad = np.tensordot(x_col, rotate_kernel(kernel), axes=[(1, 4, 5), (1, 2, 3)]).transpose(0, 3, 1, 2)
    input_grad = unwrap_padding(input_grad, padding)
    
    ## 梯度更新
    input = make_padding(input, padding)
    x_col = im2col(input, (oh, ow), dilation=stride)
    kernel_grad = np.tensordot(x_col, eta, axes=[(0, 4, 5), (0, 2, 3)]).transpose(0, 3, 1, 2)
    return input_grad, kernel_grad


input = np.arange(9).reshape(1, 1, 3, 3)
kernel = np.arange(4).reshape(*(1, 1, 2, 2))
padding = (0, 0)
stride = (1, 1)
dilation = (1, 1)
out_conv = conv(input, kernel, padding=padding, stride=stride, dilation=dilation) 
print(out_conv.shape)
eta = np.ones_like(out_conv)
backward_conv(input, kernel, eta, padding=padding, stride=stride, dilation=dilation)

(1, 1, 2, 2)


(array([[[[0., 1., 1.],
          [2., 6., 4.],
          [2., 5., 3.]]]]), array([[[[ 8, 12],
          [20, 24]]]]))

## 使用 PyTorch 验证结果

In [5]:
import torch
import torch.nn as nn
from torch.autograd import Variable

input = Variable(torch.arange(9).view(1, 1, 3, 3).float(), requires_grad=True)
net = nn.Conv2d(1, 1, 2, padding=0, stride=1, bias=False)
shape = net.weight.data.size()
net.weight.data.copy_(torch.arange(4).view(shape))
output = net(input)
print(output)
y = output.sum()
print(y)
y.backward()
print(input.grad)
print(net.weight.grad)

tensor([[[[19., 25.],
          [37., 43.]]]], grad_fn=<ThnnConv2DBackward>)
tensor(124., grad_fn=<SumBackward0>)
tensor([[[[0., 1., 1.],
          [2., 6., 4.],
          [2., 5., 3.]]]])
tensor([[[[ 8., 12.],
          [20., 24.]]]])


## 最后的思考问题

前面说过, 在 `im2col` 中有这样的判断:

```python
assert int(oh) == oh and int(ow) == ow, 'conv2d not aligned'
```

这就希望在前向传播的时候:

$h + 2p - \left(d\times\left(kh - 1\right) + 1\right)$ 能够整除 $s$, 否则抛出异常.

参考上面 "关于误差传播" 这一节, 将公式写精确应该是这样 (将余数 $n$ 也表示出来):

$$
\begin{aligned}
h + 2p - kh &= m * s + n + 1 (0 \leq n < s)\\
oh' &= \frac{h + 2*p - kh}{s} + 1 \\
(oh' - 1) * s + 1 &= h + 2p - (kh -1) \\
oh' &= m + 1 + \frac{n}{s} = oh + \frac{n}{s} \\
(oh' - 1) * s + 1 &= (oh - 1 + \frac{n}{s}) * s = (oh - 1) * s + n + 1
\end{aligned}
$$

那么代码中特征大小的变化分别是:

$$
\begin{aligned}
\text{ieta:} &\quad oh \\
\text{dilation:} &\quad nh = (oh - 1) * s + 1 \\
\text{padding:} &\quad H = nh + 2 * (kh - 1) \\
\text{im2col:} &\quad r = H - kh + 1
\end{aligned}
$$

对 $r$ 进行化简:

$$
\begin{aligned}
r &= H - (kh - 1) \\
&= nh + 2 * (kh - 1) - (kh - 1) \\
&= (oh - 1) * s + 1 + (kh - 1)  \\
&= (oh' - 1) * s + 1 - n + (kh - 1) \\
&= h + 2p - (kh -1) - n + (kh - 1) \\
&= h + 2p - n
\end{aligned}
$$

此时结果是 $h + 2p - n$, 如果最后对结果进行 `unwrap_padding`, 那么输出结果的大小就比 `h` 小了, 此时输出的梯度 `input_grad` 无法和 输入 `input` 对应.

想了一整天我也没有找到办法解决这个问题. 所以以后千万不要 "追求完美", 慢慢来, 多研究别人的代码, 最后将其理解!